In [49]:
import json
import glob
import datetime
import math
import pandas as pd
import pickle
import numpy as np
# from dataset_creator.candidate import Candidate
# from utilsAndConstants.utils import normalizeQualifications

In [6]:
EDUCATION_OR_JOB_WITH_NO_DATES = 3  # months count if you had a job that has no associated dates
EDUCATION_OR_JOB_WITH_SAME_YEAR = 6  # months count if you had a job that started and finished in the same year
EDUCATION_OR_JOB_WITH_UNDEFINED_DATES = 1  # month given that the person entered the job

In [ ]:
# need path to json files
path = '../../rawData/Xing/*.json'
entireDataSet = pd.DataFrame(columns=['protected', 'nonProtected', 'originalOrdering'])

files = glob.glob(path)
for filename in files:
    currentfile = open(filename)
    data = json.load(currentfile)

In [39]:
def readFileOfQuery(filename):
        """
        takes one .json file and reads all information, creates candidate objects from these
        information and sorts them into 3 arrays. One contains all protected candidates, one contains
        all non-protected candidates, one contains all candidates in the same order as they appear
        in the json-file

        @param filename: the json's filename

        @return:
            key: the search query string
            protected: array that contains all protected candidates
            nonProtected: array that contains all nonProtected candidates

        """
        protected = []
        nonProtected = []
        originalOrdering = []

        currentfile = open(filename)
        data = json.load(currentfile)

        xingSearchQuery = data['category']
        # if the Xing search query results in a gender neutral list,
        # we take female as the protected attribute
        protectedAttribute = 'm' if data['dominantSexXing'] == 'f' else 'f'

        for r in data['profiles']:
            # determine Member since / Hits
            if 'memberSince_Hits' in r['profile'][0]:
                hits_string = r['profile'][0]['memberSince_Hits']
                hits = hits_string.split(' / ')[1]
            else:
                hits = 1

            work_experience = determineWorkMonths(r)
            edu_experience = determineEduMonths(r)
            score = (work_experience + edu_experience) * int(hits)

            if determineIfProtected(r, protectedAttribute):
                protected.append([score, protectedAttribute])
                originalOrdering.append([score, protectedAttribute])
            else:
                nonProtected.append([score, protectedAttribute])
                originalOrdering.append([score, protectedAttribute])

#         protected.sort(key=lambda candidate: candidate.qualification, reverse=True)
#         nonProtected.sort(key=lambda candidate: candidate.qualification, reverse=True)

#         normalizeQualifications(protected + nonProtected)
#         normalizeQualifications(originalOrdering)

        currentfile.close()
        return xingSearchQuery, protected, nonProtected, originalOrdering

In [46]:
def getOriginalOrderings(filename):
        """
        takes one .json file and reads all information, creates candidate objects from these
        information and sorts them into 3 arrays. One contains all protected candidates, one contains
        all non-protected candidates, one contains all candidates in the same order as they appear
        in the json-file

        @param filename: the json's filename

        @return:
            key: the search query string
            protected: array that contains all protected candidates
            nonProtected: array that contains all nonProtected candidates

        """
        originalOrdering = []

        currentfile = open(filename)
        data = json.load(currentfile)

        xingSearchQuery = data['category']
        # if the Xing search query results in a gender neutral list,
        # we take female as the protected attribute
        protectedAttribute = 'm' if data['dominantSexXing'] == 'f' else 'f'

        for r in data['profiles']:
            # determine Member since / Hits
            if 'memberSince_Hits' in r['profile'][0]:
                hits_string = r['profile'][0]['memberSince_Hits']
                hits = hits_string.split(' / ')[1]
            else:
                hits = 1

            work_experience = determineWorkMonths(r)
            edu_experience = determineEduMonths(r)
            score = (work_experience + edu_experience) * int(hits)

            originalOrdering.append([score, protectedAttribute])

#         protected.sort(key=lambda candidate: candidate.qualification, reverse=True)
#         nonProtected.sort(key=lambda candidate: candidate.qualification, reverse=True)

#         normalizeQualifications(protected + nonProtected)
#         normalizeQualifications(originalOrdering)

        currentfile.close()
        return xingSearchQuery, originalOrdering

In [40]:
# need path to json files
path = '../../rawData/Xing/*.json'
entireDataSet = pd.DataFrame(columns=['protected', 'nonProtected', 'originalOrdering'])

files = glob.glob(path)


In [61]:
rankings = []
for filename in files:
    key, origOrder = getOriginalOrderings(filename)
    rankings.append(origOrder)
print(np.shape(rankings))

(57,)


In [62]:
# list of set of 57 rankings. Each has roughly 40 items, 
# each with an associated score and protected attribute (gender)

with open('originalRankings-only', 'wb') as handle:
    pickle.dump(rankings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [52]:
def determineIfProtected(r, protAttr):
        """
        takes a JSON profile and finds if the person belongs to the protected group

        Parameter:
        ---------
        r : JSON node
        a person description in JSON, everything below node "profile"

        """

        if 'sex' in r['profile'][0]:
            if r['profile'][0]['sex'] == protAttr:
                return True
            else:
                return False
        else:
            print('>>> undetermined\n')
            return False
        
def determineWorkMonths(r):
        """
        takes a person's profile as JSON node and computes the total amount of work months this
        person has

        Parameters:
        ----------
        r : JSON node
        """

        total_working_months = 0  # ..of that profile
        job_duration = 0

        if len(r['profile'][0]) > 4:  # a job is on the profile
            list_of_Jobs = r['profile'][0]['jobs']
            # print('profile summary' + str(r['profile'][0]['jobs']))
            for count in range(0, len(list_of_Jobs)):
                if len(list_of_Jobs[count]) > 3:  # an exact duration is given at 5 nodes!

                    job_duration_string = list_of_Jobs[count]['jobDates']
                    if job_duration_string == 'bis heute':
                        # print('job with no dates found - will be count for ' + str(job_with_no_dates) + ' months.')
                        job_duration = EDUCATION_OR_JOB_WITH_NO_DATES

                    else:
                        job_start_string, job_end_string = job_duration_string.split(' - ')

                        if len(job_start_string) == 4:
                            job_start = datetime.datetime.strptime(job_start_string, "%Y")
                        elif len(job_start_string) == 7:
                            job_start = datetime.datetime.strptime(job_start_string, "%m/%Y")
                        else:
                            print("error reading start date")

                        if len(job_end_string) == 4:
                            job_end = datetime.datetime.strptime(job_end_string, "%Y")
                        elif len(job_end_string) == 7:
                            job_end = datetime.datetime.strptime(job_end_string, "%m/%Y")
                        else:
                            print("error reading end date")

                        if job_end - job_start == 0:
                            delta = EDUCATION_OR_JOB_WITH_SAME_YEAR
                        else:
                            delta = job_end - job_start

                        job_duration = math.ceil(delta.total_seconds() / 2629743.83)

                total_working_months += job_duration
#         else:
#             print('-no jobs on profile-')

        return total_working_months


def determineEduMonths(r):
        """
        takes a person's profile as JSON node and computes the total amount of work months this
        person has

        Parameters:
        ----------
        r : JSON node
        """

        total_education_months = 0  # ..of that profile
        edu_duration = 0

        if 'education' in r:  # education info is on the profile
            list_of_edu = r['education']  # edu child nodes {institution, url, degree, eduDuration}
            # print('education summary' + str(r['education']))
            for count in range(0, len(list_of_edu)):
                if 'eduDuration' in list_of_edu[count]:  # there are education dates

                    edu_duration_string = list_of_edu[count]['eduDuration']
                    if edu_duration_string == ('bis heute' or None or ''):
                        edu_duration = EDUCATION_OR_JOB_WITH_NO_DATES
                    else:
                        edu_start_string, edu_end_string = edu_duration_string.split(' - ')

                        if len(edu_start_string) == 4:
                            edu_start = datetime.datetime.strptime(edu_start_string, "%Y")
                        elif len(edu_start_string) == 7:
                            edu_start = datetime.datetime.strptime(edu_start_string, "%m/%Y")
                        else:
                            print("error reading start date")

                        if len(edu_end_string) == 4:
                            edu_end = datetime.datetime.strptime(edu_end_string, "%Y")
                        elif len(edu_end_string) == 7:
                            edu_end = datetime.datetime.strptime(edu_end_string, "%m/%Y")
                        else:
                            print("error reading end date")

                        if edu_end - edu_start == 0:
                            delta = EDUCATION_OR_JOB_WITH_SAME_YEAR
                        else:
                            delta = edu_end - edu_start

                        edu_duration = math.ceil(delta.total_seconds() / 2629743.83)

                        # print(job_duration_string)
                        # print('this job: ' + str(job_duration))

                else: edu_duration = EDUCATION_OR_JOB_WITH_NO_DATES

                total_education_months += edu_duration
                # print('total jobs: ' + str(total_working_months))

            # print("studying: " + str(total_education_months))
#         else:
#             print('-no education on profile-')

        return total_education_months
